In [0]:
from pyspark.sql.functions import *

raw_path = "/Volumes/workspace/default/nhs_workforce/Monthly turnover from organisation benchmarking source data, November 2023.csv"

df_raw = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("/FileStore/tables/nhs_workforce_2023.csv")
)

df_raw.write.format("delta").mode("overwrite").save(raw_path)
